## Neural Networks

We can construct neural networks using the <mark style="background-color: Yellow">torch.nn</mark> package.

Now that you had a glimpse of autograd, nn depends on autograd to define models and differentiate them. An nn.Module contains layers, and a method forward(input)that returns the output.


### convnet

It is a simple feed-forward network. It takes the input,feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:
- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: <mark style="background-color: light-blue">weight = weight - learning_rate * gradient</mark>
        
### Define the network

Let's define the network:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class network(nn.Module):

    def __init__(self):
        super(network, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = mx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = network()
print(net)

network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the <mark style="background-color: yellow">forward</mark>,  and the <mark style="background-color: yellow">backward</mark> function (where gradients are computed) is automatically defined for you using autograd. You can use any of the Tensor operations in the <mark style="background-color: yellow">forward</mark> function.

#### Practice Questions
1. Try printing the number of parameters in each layer.

2. Modify the network to include dropout layers. How does it affect the output?

3. Implement a function to calculate the total number of trainable parameters.

#### Getting learnable parameters
The learnable parameters of a model are returned by <mark style="background-color: yellow">net.parameters()</mark>.

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


Let’s try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [ ]:
inp = torch.randn(1, 1, 32, 32)
out = net(inp)
print(out)

tensor([[-0.0426,  0.1086,  0.0513,  0.0206, -0.0837,  0.0527, -0.1580, -0.0314,
          0.0627, -0.1058]], grad_fn=<AddmmBackward0>)


#### Practice
1. Modify the input to have a batch size of 5 and check if the model runs correctly.

2. Change the input size to 64x64 and modify the network accordingly.

3. Implement batch normalization after each linear layer.

#### Computing Zero grad
Zero the gradient buffers of all parameters and backprops with random gradients:

In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

__Note:__
<mark style="background-color: yellow">torch.nn</mark> only supports mini-batches. The entire <mark style="background-color: yellow">torch.nn</mark> package only supports inputs that are a mini-batch of samples, and not a single sample.
For example, <mark style="background-color: yellow">nn.Conv2d</mark> will take in a 4D Tensor of <mark style="background-color: yellow">nSamples x nChannels x Height x Width</mark>.
If you have a single sample, just use <mark style="background-color: yellow">input.unsqueeze(0)</mark> to add a fake batch dimension.

__At this point, we covered:__
- Defined neutral network
- Processing input and calling backward

### Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.
There are several different loss functions under the nn package . A simple loss is: <mark style="background-color: yellow">nn.MSELoss</mark> which computes the mean-squared error between the input and the target.
    
For example:

In [ ]:
output = net(inp)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.5992, grad_fn=<MseLossBackward0>)


#### Practice
1. Change the loss function to CrossEntropyLoss and modify the target accordingly.

2. Implement a function to log the loss value at each epoch.

3. Compare different loss functions and analyze their impact on training.

Now, if you follow loss in the backward direction, using its <mark style="background-color: yellow">.grad_fn</mark> attribute, you will see a graph of computations that looks like this:

input  ->   conv2d  ->   relu  ->   maxpool2d  ->   conv2d ->   relu  ->   maxpool2d
      
      -> view -> linear -> relu -> linear -> relu -> linear
      
      -> MSELoss
      
      -> loss

So, when we call <mark style="background-color: yellow">loss.backward()</mark>, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has requires_grad=True will have their .grad Tensor accumulated with the gradient.


For illustration, let us follow a few steps <mark style="background-color: yellow">backward</mark>:

In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### Backprop

To backpropagate the error all we have to do is to <mark style="background-color: yellow">loss.backward()</mark>. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.


Now we shall call <mark style="background-color: yellow">loss.backward()</mark>, and have a look at conv1’s bias gradients before and after the backward.

In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0039, -0.0066,  0.0013,  0.0068, -0.0038, -0.0091])


#Practice Question
1. Implement a function to visualize gradient flow in the model.

2. Modify the loss.backward() call to accumulate gradients over multiple batches.

3. Check if using loss.backward(retain_graph=True) affects training.

### Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):
    
   <mark style="background-color: yellow"> weight = weight - learning_rate * gradient </mark>
   
We can implement this using simple Python code:

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

#### Practice Questions
1. Modify the learning rate and observe its impact on convergence.

2. Implement an adaptive learning rate that decreases over epochs.

3. Compare different optimizers like Adam and RMSprop on the same model.

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(inp)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

Observe how gradient buffers had to be manually set to zero using <mark style="background-color:yellow">optimizer.zero_grad()</mark>. This is because gradients are accumulated as explained in the <mark style="font-color:red">Backprop</mark> section.

### Training a Neural Network on Dummy MNIST

-  Here we demonstrate training a simple feedforward network on a subset of the MNIST dataset.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Define transformations: convert images to tensors and normalize.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download the MNIST training dataset.
mnist_full = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# For quick demonstration, we use a subset (e.g., first 1000 samples).
subset_indices = list(range(1000))
mnist_subset = Subset(mnist_full, subset_indices)
# Create a DataLoader for batching.
train_loader = DataLoader(mnist_subset, batch_size=64, shuffle=True)

# Define a simple feedforward network (MLP) for MNIST classification.
class SimpleMNISTNet(nn.Module):
    def __init__(self):
        super(SimpleMNISTNet, self).__init__()
        # Input size: 28*28 = 784. Two hidden layers, and 10 outputs for the digits.
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # Flatten the image tensor.
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the network, define loss and optimizer.
mnist_model = SimpleMNISTNet()
criterion_mnist = nn.CrossEntropyLoss()
optimizer_mnist = optim.SGD(mnist_model.parameters(), lr=0.01)

# Training loop for a few epochs (for demonstration purposes, we use 2 epochs).
num_epochs = 2
print("\nStarting training on MNIST subset...")
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer_mnist.zero_grad()   # Clear gradients.
        outputs = mnist_model(data)     # Forward pass.
        loss = criterion_mnist(outputs, target)  # Compute loss.
        loss.backward()                 # Backward pass.
        optimizer_mnist.step()          # Update weights.
        running_loss += loss.item()
        if (batch_idx + 1) % 10 == 0:
            avg_loss = running_loss / 10
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}], Loss: {avg_loss:.4f}')
            running_loss = 0.0

print("MNIST training complete!")


Starting training on MNIST subset...
Epoch [1/2], Batch [10], Loss: 2.3012
Epoch [2/2], Batch [10], Loss: 2.2612
MNIST training complete!


### Practice Questions
1. Modify the batch size and analyze its impact on training.

2. Implement early stopping in the training loop.

3. Use a validation split and track validation accuracy after each epoch.